In [ ]:
import pandas as pd
import numpy as np

path = "../../d/Global_Mobility_Report.csv.xz"
df = pd.read_csv(path, parse_dates=["date"])

In [ ]:
subs = {"Czechia": 'Czech', "United Kingdom": "England"}

# Filter to sub-national 
subnational_row = lambda df : df[df.sub_region_1.notna() & df.sub_region_2.isna() & df.metro_area.isna()]
snat = subnational_row(df)
snat = snat.drop("parks_percent_change_from_baseline", axis=1)
snat = snat.drop(["country_region_code", "sub_region_2", "metro_area", "iso_3166_2_code", "census_fips_code", "place_id"], axis=1)

snat["country_region"] = snat["country_region"].replace(subs)
snat.rename(columns={"date":"Date", "country_region": "Country", "sub_region_1": "Area"}, inplace=True)
#nat["Date"] = pd.to_datetime(nat["Date"], utc=True)
snat.set_index(["Country", "Area", "Date"])

# Filter to national 
subnational_row = lambda df : df[df.sub_region_1.isna() & df.sub_region_2.isna() & df.metro_area.isna()]
nat = subnational_row(df)
nat = nat.drop("parks_percent_change_from_baseline", axis=1)
nat = nat.drop(["country_region_code", "sub_region_2", "metro_area", "iso_3166_2_code", "census_fips_code", "place_id"], axis=1)

nat["country_region"] = nat["country_region"].replace(subs)
nat.rename(columns={"date":"Date", "country_region": "Country", "sub_region_1": "Area"}, inplace=True)
#nat["Date"] = pd.to_datetime(nat["Date"], utc=True)
nat.set_index(["Country", "Area", "Date"])
len(nat), len(snat)


In [ ]:
dfb = pd.read_csv("../../d/modelSharma_dataSharma.csv", parse_dates=["Date"])

dfb = dfb.merge(nat, on=["Country", "Date"], how="left", suffixes=('', '_'))
#dfb.fillna(0.0, inplace=True)
MOBS = {
    "retail_and_recreation_percent_change_from_baseline": "Recre",
    "grocery_and_pharmacy_percent_change_from_baseline": "Groc",
    "transit_stations_percent_change_from_baseline": "Trans",
    "workplaces_percent_change_from_baseline": "Work",
    "residential_percent_change_from_baseline": "Resid",
    }
mobility = np.mean([np.float32(dfb[m]) for m in MOBS], axis=0)
mobility
print(sum(dfb["retail_and_recreation_percent_change_from_baseline"].isna()), len(dfb))
dfb.groupby("Country").apply(lambda d: sum(d["retail_and_recreation_percent_change_from_baseline"].isna()) / len(d))
dfb.drop(MOBS, inplace=True, axis=1)
dfb.drop(["Area_"], inplace=True, axis=1)
dfb["Mobility decrease"] = mobility / -100.0
dfb.to_csv(f"../../d/modelSharma_dataSharma_countryMobility_{''.join(MOBS.values())}.csv", index=False)
dfb